In [227]:
!pip install scikit-learn
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [228]:
import sklearn
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler

In [229]:
#convert the txt files to csv files
directories = ['heart_rate','motion','steps','labels']

for d in directories:
    for name in os.listdir(d):
        with open(os.path.join(d, name)) as f:
             file_path = os.path.join(d, name)
             dir_name = f"{d}_csv"
             os.makedirs(dir_name,exist_ok=True)
             df1 = pd.read_csv(file_path)
             csv_file_path = os.path.join(dir_name, f"{os.path.splitext(name)[0]}")
             df1.to_csv(csv_file_path + '.csv', 
                      index = None)

In [230]:
#sanity check 
directories = ['steps_csv', 'heart_rate_csv', 'labels_csv', 'motion_csv']
counters = {'steps_csv': 0, 'heart_rate_csv': 0, 'labels_csv': 0, 'motion_csv': 0}
for d in directories:
    for name in os.listdir(d):
        counters[d] += 1

for key, value in counters.items():
    print(f"{key}: {value} files found")        

steps_csv: 31 files found
heart_rate_csv: 31 files found
labels_csv: 31 files found
motion_csv: 31 files found


In [231]:
file_path = "motion_csv/2598705_acceleration.csv"
with open(file_path, 'r') as f:
    for _ in range(5):
        print(f.readline())

-1781.78196907 0.2240143 -0.3001709 -0.915863

-1781.69951391 0.2215729 -0.2962646 -0.9193726

-1781.69724894 0.2177124 -0.2957916 -0.929245

-1781.69525695 0.2167511 -0.294342 -0.9327087

-1781.69442797 0.2177124 -0.2899323 -0.9317322



In [232]:
#outliers using IQR Method 
count = {'steps_csv':0, 'heart_rate_csv':0, 'labels_csv':0, 'motion_csv':0}
def outliers(input_folder,threshold):
     df_outliers = []
     for name in os.listdir(input_folder):
        if not name.lower().endswith('.csv'):
            continue
        
        file_path = os.path.join(input_folder, name)

        if (input_folder == 'motion_csv'):
            df = pd.read_csv(file_path, header=None, sep=r'\s+')
        else:
            df = pd.read_csv(file_path, header=None)
            
        value_columns = df.iloc[:, 1:]
        
        mask = pd.Series(True, index=df.index)

        for col in value_columns.columns:
            Q1 = value_columns[col].quantile(0.25)
            Q3 = value_columns[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            mask &= (value_columns[col] >= lower_bound) & (value_columns[col] <= upper_bound)  
            
        df_cleaned = df[mask]
        df_outliers.append((input_folder, name, df_cleaned)) 
        
        count[input_folder] +=1

     return df_outliers
            

In [244]:
#to be rewriten 
def normalize(df):


In [245]:
folders = ['steps_csv', 'heart_rate_csv', 'motion_csv']
threshold = 1.5

all_outliers = []

for folder in folders:
    df_list = outliers(folder, threshold)
    all_outliers.extend(df_list)

for folder, filename, df in all_outliers:
    result = normalize(df)
    
for folder, filename, df in all_outliers:
    print(f"Folder: {folder} | File: {filename} | Shape after cleaning: {df.shape}")
    print(f"Normalized {folder}/{filename} | Shape: {result.shape}")


Folder: steps_csv | File: 3997827_steps.csv | Shape after cleaning: (850, 2)
Normalized steps_csv/3997827_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 46343_steps.csv | Shape after cleaning: (1119, 2)
Normalized steps_csv/46343_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 9618981_steps.csv | Shape after cleaning: (517, 2)
Normalized steps_csv/9618981_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 2598705_steps.csv | Shape after cleaning: (1035, 2)
Normalized steps_csv/2598705_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 6220552_steps.csv | Shape after cleaning: (864, 2)
Normalized steps_csv/6220552_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 4314139_steps.csv | Shape after cleaning: (927, 2)
Normalized steps_csv/4314139_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 1066528_steps.csv | Shape after cleaning: (312, 2)
Normalized steps_csv/1066528_steps.csv | Shape: (1393470, 4)
Folder: steps_csv | File: 759667_ste

In [246]:
for key,value in count.items():
     print(f"{key}:{value} files")

steps_csv:186 files
heart_rate_csv:186 files
labels_csv:0 files
motion_csv:186 files


In [179]:
for d in all_outliers:
    x,y = d.shape
    if x > 0 or y > 0 : 
        z = True
    else:
        z = False

print(z)

True
